In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid")
from scipy import stats
from scipy.stats import norm
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
train= pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test= pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
#descriptive
train['SalePrice'].describe()

In [ ]:
#histogram
sns.distplot(train['SalePrice'])

In [ ]:
#correlation matrix
corl=train.corr()
f , ax=plt.subplots(figsize=(20,10))
sns.heatmap(corl , linewidths=.5,cmap='Dark2_r')

In [ ]:
train.corr()['SalePrice'].sort_values(ascending=False)[1:]

In [ ]:
cols=['OverallQual' ,'GrLivArea'      
,'GarageCars'      
,'GarageArea'       
,'TotalBsmtSF'      
,'1stFlrSF'         
,'FullBath'         
,'TotRmsAbvGrd'     
,'YearBuilt'        
,'YearRemodAdd'     
,'GarageYrBlt'      
,'MasVnrArea'       
,'Fireplaces'       
,'BsmtFinSF1'       
,'LotFrontage'      
,'WoodDeckSF'      
,'2ndFlrSF'        
,'OpenPorchSF'    
,'HalfBath'         
,'LotArea'     
,'BsmtFullBath','BsmtUnfSF'        
,'BedroomAbvGr'     
,'ScreenPorch'      
,'PoolArea'         
,'MoSold'           
,'3SsnPorch']

In [ ]:
X=train[cols]
X_test=test[cols]

In [ ]:
X.head()

In [ ]:
X.isnull().sum()

In [ ]:
X.corr()['LotFrontage'].sort_values(ascending=False)[1:]

In [ ]:
#I try Regression missing value LotFrontage 
LFR=X[['LotFrontage','1stFlrSF','LotArea','GrLivArea','TotalBsmtSF']].copy()

In [ ]:
LFR

In [ ]:
LFR_null=LFR[LFR['LotFrontage'].isna()]

In [ ]:
LFR_null

In [ ]:
LFR=LFR.dropna()

In [ ]:
import xgboost as xb

In [ ]:
reg=xb.XGBRegressor()

In [ ]:
reg.fit(LFR[['1stFlrSF','LotArea','GrLivArea','TotalBsmtSF']],LFR['LotFrontage'])

In [ ]:
reg.score(LFR[['1stFlrSF','LotArea','GrLivArea','TotalBsmtSF']],LFR['LotFrontage'])

In [ ]:
LFR_null_pred=reg.predict(LFR_null[['1stFlrSF','LotArea','GrLivArea','TotalBsmtSF']])

In [ ]:
LFR_null['LotFrontage']=LFR_null_pred

In [ ]:
LFR_finall=pd.concat([LFR,LFR_null],axis=0)

In [ ]:
X[['LotFrontage','1stFlrSF','LotArea','GrLivArea','TotalBsmtSF']]=LFR_finall

In [ ]:
X.isnull().sum()

In [ ]:
X['GarageYrBlt']=X['GarageYrBlt'].fillna(0)
X['MasVnrArea']=X['MasVnrArea'].fillna(0)

In [ ]:
X['SalePrice']=train['SalePrice']

In [ ]:
X.plot.scatter(x='GarageArea',y='SalePrice',color='r')

In [ ]:
X.drop(X[X['GarageArea']>1200].index,inplace=True)

In [ ]:
X.plot.scatter(x='GrLivArea',y='SalePrice',color='r')

In [ ]:
X.drop(X[(X['GrLivArea'] >3000) & (X['SalePrice'] <400000)].index,inplace=True)

In [ ]:
X.plot.scatter(x='LotArea',y='SalePrice',color='r')

In [ ]:
X.drop(X[(X['LotArea'] >50000) & (X['SalePrice'] <400000)].index,inplace=True)

In [ ]:
X.plot.scatter(x='YearBuilt',y='SalePrice',color='r')

In [ ]:
X.plot.scatter(x='BsmtUnfSF',y='SalePrice',color='r')

In [ ]:
X.drop(X[X['GarageArea']>1800].index,inplace=True)

In [ ]:
X.plot.scatter(x='1stFlrSF',y='SalePrice',color='r')

In [ ]:
X.drop(X[X['1stFlrSF']>2600].index,inplace=True)

In [ ]:
X.plot.scatter(x='1stFlrSF',y='SalePrice',color='r')

In [ ]:
X.plot.scatter(x='BsmtUnfSF',y='SalePrice',color='r')

In [ ]:
X.drop(X[X['1stFlrSF']>2000].index,inplace=True)

In [ ]:
X.plot.scatter(x='BsmtUnfSF',y='SalePrice',color='r')

In [ ]:
X_test.isnull().sum()

In [ ]:
X_test['TotalBsmtSF'].describe()

In [ ]:
X_test.loc[X_test['TotalBsmtSF'].isnull(),'TotalBsmtSF'] = X_test['TotalBsmtSF'].mean()

In [ ]:
LFR=X_test[['LotFrontage','1stFlrSF','LotArea','GrLivArea','TotalBsmtSF']].copy()
LFR_null=LFR[LFR['LotFrontage'].isna()]
LFR=LFR.dropna()
LFR_null_pred=reg.predict(LFR_null[['1stFlrSF','LotArea','GrLivArea','TotalBsmtSF']])
LFR_null['LotFrontage']=LFR_null_pred
LFR_finall=pd.concat([LFR,LFR_null],axis=0)

In [ ]:
X_test[['LotFrontage','1stFlrSF','LotArea','GrLivArea','TotalBsmtSF']]=LFR_finall

In [ ]:
X_test['LotFrontage'].isnull().sum()

In [ ]:
X_test['BsmtFullBath'].describe()

In [ ]:
X_test['BsmtFullBath']=X_test['BsmtFullBath'].fillna(0)

In [ ]:
X_test['MasVnrArea'].describe()

In [ ]:
X_test['MasVnrArea']=X_test['MasVnrArea'].fillna(0)

In [ ]:
X_test.loc[X_test['BsmtUnfSF'].isnull(),'BsmtUnfSF'] = X_test['BsmtUnfSF'].mean()

In [ ]:
X_test.loc[X_test['GarageCars'].isnull()].index

In [ ]:
X_test.loc[X_test['GarageArea'].isnull()].index

In [ ]:
X_test['GarageCars']=X_test['GarageCars'].fillna(0)
X_test['GarageArea']=X_test['GarageArea'].fillna(0)

In [ ]:
X_test.loc[X_test['BsmtFinSF1'].isnull(),'BsmtFinSF1'] = X_test['BsmtFinSF1'].mean()

In [ ]:
X_test.loc[X_test['GarageYrBlt'].isnull()].index

In [ ]:
X_test['GarageYrBlt']=X_test['GarageYrBlt'].fillna(0)

In [ ]:
sns.distplot(X['SalePrice'], fit=norm);
fig = plt.figure()
res = stats.probplot(X['SalePrice'], plot=plt)

In [ ]:
X['SalePrice']=np.log(X['SalePrice'])

In [ ]:
sns.distplot(X['SalePrice'], fit=norm);
fig = plt.figure()
res = stats.probplot(X['SalePrice'], plot=plt)

In [ ]:
sns.distplot(X['BsmtUnfSF'], fit=norm);
fig = plt.figure()
res = stats.probplot(X['BsmtUnfSF'], plot=plt)

In [ ]:
#convert 0 to 1 because log(0)=-inf
X.loc[X['BsmtUnfSF']==0,'BsmtUnfSF']=1
X['BsmtUnfSF']=np.log(X['BsmtUnfSF'])
sns.distplot(X['BsmtUnfSF'], fit=norm);
fig = plt.figure()
res = stats.probplot(X[X['BsmtUnfSF']>0]['BsmtUnfSF'], plot=plt)

In [ ]:
sns.distplot(X['GrLivArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(X['GrLivArea'], plot=plt)

In [ ]:
X['GrLivArea']=np.log(X['GrLivArea'])
sns.distplot(X['GrLivArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(X['GrLivArea'], plot=plt)

In [ ]:
sns.distplot(X['GarageArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(X['GarageArea'], plot=plt)

In [ ]:
X.loc[X['GarageArea']==0,'GarageArea']=1
X['GarageArea']=np.log(X['GarageArea'])
sns.distplot(X[X['GarageArea']>0]['GarageArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(X[X['GarageArea']>0]['GarageArea'], plot=plt)

In [ ]:
sns.distplot(X['TotalBsmtSF'], fit=norm);
fig = plt.figure()
res = stats.probplot(X['TotalBsmtSF'], plot=plt)

In [ ]:
X.loc[X['TotalBsmtSF']==0,'TotalBsmtSF']=1
X['TotalBsmtSF']=np.log(X['TotalBsmtSF'])
sns.distplot(X[X['TotalBsmtSF']>0]['TotalBsmtSF'], fit=norm);
fig = plt.figure()
res = stats.probplot(X[X['TotalBsmtSF']>0]['TotalBsmtSF'], plot=plt)

In [ ]:
sns.distplot(X['1stFlrSF'],fit=norm)
fig=plt.figure()
res=stats.probplot(X['1stFlrSF'],plot=plt)

In [ ]:
X['1stFlrSF']=np.log(X['1stFlrSF'])
sns.distplot(X['1stFlrSF'],fit=norm)
fig=plt.figure()
res=stats.probplot(X['1stFlrSF'],plot=plt)

In [ ]:
sns.distplot(X['LotArea'],fit=norm)
fig=plt.figure()
res=stats.probplot(X['LotArea'],plot=plt)

In [ ]:
X['LotArea']=np.log(X['LotArea'])
sns.distplot(X['LotArea'],fit=norm)
fig=plt.figure()
res=stats.probplot(X['LotArea'],plot=plt)

In [ ]:
sns.distplot(X['PoolArea'],fit=norm)
fig=plt.figure()
res=stats.probplot(X['PoolArea'],plot=plt)

In [ ]:
X.loc[X['PoolArea']==0,'PoolArea']=1
X['PoolArea']=np.log(X['PoolArea'])
sns.distplot(X[X['PoolArea']>0]['PoolArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(X[X['PoolArea']>0]['PoolArea'], plot=plt)

In [ ]:
sns.distplot(X['ScreenPorch'],fit=norm)
fig=plt.figure()
res=stats.probplot(X['ScreenPorch'],plot=plt)

In [ ]:
X.loc[X['ScreenPorch']==0,'ScreenPorch']=1
X['ScreenPorch']=np.log(X['ScreenPorch'])
sns.distplot(X[X['ScreenPorch']>0]['ScreenPorch'], fit=norm);
fig = plt.figure()
res = stats.probplot(X[X['ScreenPorch']>0]['ScreenPorch'], plot=plt)

In [ ]:
sns.distplot(X['2ndFlrSF'],fit=norm)
fig=plt.figure()
res=stats.probplot(X['2ndFlrSF'],plot=plt)

In [ ]:
X.loc[X['2ndFlrSF']==0,'2ndFlrSF']=1
X['2ndFlrSF']=np.log(X['2ndFlrSF'])
sns.distplot(X[X['2ndFlrSF']>0]['2ndFlrSF'], fit=norm);
fig = plt.figure()
res = stats.probplot(X[X['2ndFlrSF']>0]['2ndFlrSF'], plot=plt)

In [ ]:
sns.distplot(X['OpenPorchSF'],fit=norm)
fig=plt.figure()
res=stats.probplot(X['OpenPorchSF'],plot=plt)

In [ ]:
X.loc[X['OpenPorchSF']==0,'OpenPorchSF']=1
X['OpenPorchSF']=np.log(X['OpenPorchSF'])
sns.distplot(X[X['OpenPorchSF']>0]['OpenPorchSF'], fit=norm);
fig = plt.figure()
res = stats.probplot(X[X['OpenPorchSF']>0]['OpenPorchSF'], plot=plt)

In [ ]:
X.describe()

In [ ]:
X.loc[X['3SsnPorch']==0,'3SsnPorch']=1
X['3SsnPorch']=np.log(X['3SsnPorch'])
sns.distplot(X[X['3SsnPorch']>0]['3SsnPorch'], fit=norm);
fig = plt.figure()
res = stats.probplot(X[X['3SsnPorch']>0]['3SsnPorch'], plot=plt)

In [ ]:
sns.distplot(X['YearBuilt'],fit=norm)
fig=plt.figure()
res=stats.probplot(X['YearBuilt'],plot=plt)

In [ ]:
X['YearBuilt']=np.log(X['YearBuilt'])
X['YearRemodAdd']=np.log(X['YearRemodAdd'])

In [ ]:
y=X['SalePrice']
X.drop(['SalePrice'],axis=1,inplace=True)

In [ ]:
reg=xb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.5,
             colsample_bynode=0.5, colsample_bytree=0.5, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.100000012,
             max_delta_step=3, max_depth=12, min_child_weight=3, 
             monotone_constraints='()', n_estimators=1000, n_jobs=-1,
             num_parallel_tree=1, predictor='auto', random_state=42, reg_alpha=0.0015,
             reg_lambda=0.01, scale_pos_weight=0.2, subsample=1, tree_method='exact',
             validate_parameters=0.4, verbosity=None)

In [ ]:
reg.fit(X,y)

In [ ]:
reg.score(X,y)

In [ ]:
X_test.describe()

In [ ]:
X_test.loc[X_test['BsmtUnfSF']==0,'BsmtUnfSF']=1
X_test.loc[X_test['LotArea']==0,'LotArea']=1
X_test.loc[X_test['GrLivArea']==0,'GrLivArea']=1
X_test.loc[X_test['GarageArea']==0,'GarageArea']=1
X_test.loc[X_test['TotalBsmtSF']==0,'TotalBsmtSF']=1
X_test.loc[X_test['1stFlrSF']==0,'1stFlrSF']=1
X_test.loc[X_test['2ndFlrSF']==0,'2ndFlrSF']=1
X_test.loc[X_test['ScreenPorch']==0,'ScreenPorch']=1
X_test.loc[X_test['PoolArea']==0,'PoolArea']=1
X_test.loc[X_test['OpenPorchSF']==0,'OpenPorchSF']=1

In [ ]:
X_test['BsmtUnfSF']=np.log(X_test['BsmtUnfSF'])
X_test['LotArea']=np.log(X_test['LotArea'])
X_test['GrLivArea']=np.log(X_test['GrLivArea'])
X_test['GarageArea']=np.log(X_test['GarageArea'])
X_test['TotalBsmtSF']=np.log(X_test['TotalBsmtSF'])
X_test['1stFlrSF']=np.log(X_test['1stFlrSF'])
X_test['2ndFlrSF']=np.log(X_test['2ndFlrSF'])
X_test['ScreenPorch']=np.log(X_test['ScreenPorch'])
X_test['PoolArea']=np.log(X_test['PoolArea'])
X_test['OpenPorchSF']=np.log(X_test['OpenPorchSF'])
X_test['YearBuilt']=np.log(X_test['YearBuilt'])
X_test['YearRemodAdd']=np.log(X_test['YearRemodAdd'])